In [35]:
# https://wikidocs.net/60691

# 02. IMDB 리뷰 감성 분류하기(IMDB Movie Review Sentiment Analysis)

참고 논문: http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf

In [36]:
# 영화 사이트 IMDB의 리뷰 데이터를 이용해 1 긍정 , 0 부정을 분류해보자

In [37]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.legacy import data, datasets
import random

In [38]:
# 랜덤시드 고정
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

In [39]:
# 하이퍼파라미터 변수
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10

In [40]:
# 디바이스 설정
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("cpu와 cuda 중 다음 기기로 학습함:", DEVICE)

cpu와 cuda 중 다음 기기로 학습함: cuda


### 2. 토치텍스트를 이용한 전처리
---

In [55]:
# 1. 데이터 로드하기 : torchtext.data
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
# sequential: 순차적 데이터
# batch_first: 신경망에 입력되는 텐서의 첫번째 차원값이 batch_size가 되도록
# lower: 입력되는 모든 알파벳이 소문자가 되도록
LABEL = data.Field(sequential=False, batch_first=True)

In [58]:
# 2. 데이터 로드 및 분할하기 : torchtext.datasets

# 전체 데이터를 훈련, 테스트로 나누기(8:2)
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

In [47]:
print('trainset의 구성 요소 출력: ', trainset.fields)
print('testset의 구성 요소 출력: ', testset.fields)

trainset의 구성 요소 출력:  {'text': <torchtext.legacy.data.field.Field object at 0x0000025897174070>, 'label': <torchtext.legacy.data.field.Field object at 0x0000025897174820>}
testset의 구성 요소 출력:  {'text': <torchtext.legacy.data.field.Field object at 0x0000025897174070>, 'label': <torchtext.legacy.data.field.Field object at 0x0000025897174820>}


In [48]:
# 첫번째 훈련 샘플과 해당 샘플에 대한 레이블을 함께 출력
print(vars(trainset[0]))

IndexError: list index out of range

In [50]:
TEXT.build_vocab(trainset, min_freq=5) # 단어 집합 생성
LABEL.build_vocab(trainset)

In [52]:
vocab_size = len(TEXT.vocab)
n_classes = 2
print('단어 집합의 크기 : {}'.format(vocab_size))
print('클래스의 개수 : {}'.format(n_classes))

단어 집합의 크기 : 2
클래스의 개수 : 2
